In [ ]:
# run this if game crashes and you want to see documentation with shift/tab
import pygame
import numpy as np
import matplotlib.pyplot as plt
import time
import random


# RL Tetris Project Outline:
0) Clean up code
    - separate classes from main loop and RL components (easier to run 1 cell at the moment)
    - remove legacy comments
    - make variables any magic numbers (mainly coordinates)
    - make production ready
1) [x] Create figure generator
    - [x] Choose one of 7 blocks (O, I, S, Z, J, L, T)
        - [x] Figure out color for block
        - [x] List out rotations for each block
2) Generate board/figure
    - [x] Adjust general layout for board
        - [x] timer
        - [x] score display
        - [x] lines display
        - level display 
    - Implement buttons (not essential atm)
        - new game button (esc for now)
        - reset button
        - pause button
    - Implement debugger/ replay buff
        - back (last n = 5 blocks, and game states, probably useful for debugging)
        * need to store matrix, queue, swap piece and time? Reset all. Not sure how to deal with other saved states?
    - [x] area for next blocks (show up to k=3)
    - [x] area for swapping blocks
3) Game logic
    - clean up timer for piece movement
    - [x] set up block shadow where piece will land (useful for computer mode)
    - Set up controls for piece movement
        - [x] human mode: button presses
        - [x] computer mode (render or not)
    - [x] make piece generator function
        - [x] go to next piece queue to display
        - [x] previous next piece gets bumped to board (above game)
    - [x] make movement functions
        - [x] left, right, soft drop, hard drop
        - [x] CW rotate, CCW rotate
        - [x] kick checks for rotations (only checked example shown here: https://tetris.fandom.com/wiki/SRShttps://tetris.fandom.com/wiki/SRS, HARD to check (I'm not a skilled Tetris player)!
    - update screen function
        - [x] move piece
        - check for silver/golden squares
        - check for line clears
            - [x] drop pieces above (already implemented in original code)
            - [x] adjust score (could be updated)
            - [x] adjust lines
            - adjust level
            - play sound
           
4) RL Algorithm
    - review articles on making tetris and how to avoid getting stuck with delayed reward
    - research if possible to help model train by giving examples (showing how to clear lines, probably slow)
    - [x] create option for human input and computer with rendering (allow policy to be chosen, random or model)
        - [x] Make computer play random moves sampling from action space of allowable moves
        - Eventually swap to play best model moves and render
        - Develop algorithm to figure out 'all' (maybe most since getting every landing seems super difficult) landing spots and moves to get there
    - Create OpenAI gym tetris environment
    - Train model
        - Test different action spaces
            - Simple movements
            - complex movements
        - test different algorithms
            - double q
            - PPO
            - add more here...
        - test if adding heuristics as features improves convergence
    - PROBABLY MUCH MORE TO DO IN THIS SECTION
5) Streamlit App bonus
    - allow user to play vs computer


In [10]:
# base human game working through timer


### Code originally taken from:
# https://levelup.gitconnected.com/writing-tetris-in-python-2a16bddb5318
# highly modified
import pygame
import random
import sys
import tetris
import gym

Tetris = tetris.Tetris


# Global variables
# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (128, 128, 128)
TRANSPARENCY = 40 # out of 255
        
# colors order is linked to figure order in figure.py, this seems like a potential problem
colors = [
    (255, 255, 255), # white for empty
    (0, 255, 255), # I- Cyan
    (0, 0, 255), # J - Blue
    (255, 127, 0), # L - Orange
    (255, 255, 0), # O - Yellow
    (0, 255, 0), # S - Green 
    (128, 0, 128), # T - Purple
    (255, 0, 0), # Z - Red
    (255, 215, 0), # GOLD, not implemented
    (194, 189, 176), # SILVER, not implemented
]

class Game_env:
    size = (800, 800)
    done = False
    clock = pygame.time.Clock()
    

    
    def __init__(self, render = True, episodes = None, max_something = None):
        # Initialize the game engine
        pygame.quit() # in case it didn't properly close
        pygame.init()
        self.game = Tetris(player = 0) # probably should be passed in 
        self.render = render
        if render:
            self.screen = pygame.display.set_mode(self.size)
            pygame.display.set_caption("Tetris")
        self.game_start_time = pygame.time.get_ticks() # game start time
        self.counter = 0
        self.pressing_down = False
        self.pressing_sideways = 0
        self.cur_time = 0
        self.play_game()


    # One random function- BETTER SPOT FOR THIS GUY?    
    # Transparency for block shadow, code taken from: https://stackoverflow.com/questions/6339057/draw-a-transparent-rectangles-and-polygons-in-pygame
    def draw_rect_alpha(self,surface, color, rect):
        shape_surf = pygame.Surface(pygame.Rect(rect).size, pygame.SRCALPHA)
        pygame.draw.rect(shape_surf, color, shape_surf.get_rect())
        surface.blit(shape_surf, rect)

    def play_game(self):
        while not self.done: # also check epsisode count < desired number

            # Tetris property I THINK, this whole loop!
            self.counter += 1
            if self.counter > 100000:
                self.counter = 0
            # calculate reward
            # dividing by time doesn't work without rendering- points go extremely high
            self.game.reward = (self.game.score + self.game.landed_blocks) / ((self.game.landed_blocks+1)/4) 
               # / ((pygame.time.get_ticks()-game.game_start_time)/1000) 

            if self.game.player == 1 and self.game.state == 'start': # computer play
                # choose random action 
                actions = ['no_op', 'left', 'right', 'down', 'hard', 'cw', 'ccw', 'swap']
                act = random.sample(actions,1)[0]
                pressing_sideways = 0

                if act == actions[0]:
                    pass
                elif act == actions[1]:
                    self.game.go_side(-1)
                elif act == actions[2]:
                    self.game.go_side(1)        
                elif act == actions[3]:
                    self.game.go_down()
                # elif act == actions[4]:
                #     self.game.go_space()
                elif act == actions[5]:
                    self.game.rotate(direction = 1)
                elif act == actions[6]:
                    self.game.rotate(direction = -1)
                elif act == actions[7]:
                    self.game.swap()

            # Every time counter gets to ~ 60//level//2 (or down key)
            # drop piece (TOTALLY WRONG! FIX, somehow works though)
            # if game.state == 'start' and (counter % (fps // game.level // 2) == 0 or pressing_down):
            #     game.go_down()


            if self.game.state == 'start' and (self.counter% self.game.frames_per_drop == 0 or self.pressing_down):
                self.game.go_down()
                self.counter = 0




            # TOO FAST
            if self.pressing_sideways != 0: 
                if self.game.player == 1 or self.counter%2 == 0: 
                    self.game.go_side(self.pressing_sideways)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.done = True

                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_q:  
                        self.game.__init__(player = self.game.player)

                    # Don't attempt a player change if not rendering, only for training
                    if event.key == pygame.K_p and self.render == True:
                        self.game.change_player = True

                    if self.game.player == 1:
                        break # should exit FOR loop if computer playing, don't take inputs



                    if event.key == pygame.K_l:
                        self.game.lines += 1
                        if self.game.lines % self.game.lines_per_level == 0:
                            self.game.level += 1
                            self.game.frames_per_drop = self.game.level_frames_per_drop[min(self.game.level, self.game.max_level)]

                    if self.game.state == 'gameover':
                        self.pressing_down = False
                        self.pressing_sideways = 0
                        break
                    if event.key == pygame.K_RSHIFT:
                        self.game.rotate(direction = 1)
                    if event.key == pygame.K_SLASH:
                        self.game.rotate(direction = -1)
                    if event.key == pygame.K_DOWN:
                        self.pressing_down = True
                    else:
                        self.pressing_down = False  
                    if event.key == pygame.K_LEFT:
                        self.pressing_sideways = -1
                    elif event.key == pygame.K_RIGHT:
                        self.pressing_sideways = 1
                    else:
                        self.pressing_sideways = 0
                    if event.key == pygame.K_SPACE:
                        self.game.go_space()         
                    if event.key in [pygame.K_0, pygame.K_1, pygame.K_2, 
                                     pygame.K_3, pygame.K_4, pygame.K_5, pygame.K_6]:
                        self.game.new_figure(mode = event.key - pygame.K_0) # ASSUME K_0 is 48 and rest of numbers go up by 1            
                    if event.key == pygame.K_s:
                        self.game.swap()          
            if event.type == pygame.KEYUP:

                    if event.key == pygame.K_DOWN:
                        self.pressing_down = False
                    if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                        self.pressing_sideways = 0
                    if event.key == pygame.K_p and self.game.change_player: # change players
                        self.game.player = (self.game.player + 1) % 2
                        self.game.change_player = False



            if not(self.render):
                self.clock.tick(self.game.fps)
                continue
            self.screen.fill(WHITE)

            # Drawing screen
            for i in range(self.game.buffer, self.game.full_height):
                for j in range(self.game.width):
                    # WHAT IS ZOOM of 20 doing for the rectangle drawing
                    pygame.draw.rect(self.screen, GRAY, [self.game.x + self.game.zoom * j, 
                                                         self.game.y + self.game.zoom * i, 
                                                         self.game.zoom, self.game.zoom], 
                                     width = 3)
                    if self.game.board[i][j] > 0 and i >= self.game.buffer:
                        pygame.draw.rect(self.screen, colors[self.game.board[i][j]],
                                         [self.game.x + self.game.zoom * j + 1, 
                                          self.game.y + self.game.zoom * i + 1, 
                                          self.game.zoom - 2, self.game.zoom - 1])

            # UPDATE#
            if self.game.figure is not None:
                shadow_y = self.game.shadow_height()
                for ind in self.game.figure.image():
                    i = ind//4
                    j = ind%4
                    p = i * 4 + j

                    # Plotting of shadow piece
                    if shadow_y + i >= self.game.buffer:
                        self.draw_rect_alpha(self.screen, 
                                        tuple(list(colors[self.game.figure.type])+[TRANSPARENCY]),
                                         [self.game.x + self.game.zoom * (j + self.game.figure.x) + 1,
                                          self.game.y + self.game.zoom * (i + shadow_y) + 1,
                                          self.game.zoom - 2, self.game.zoom - 2])

                    # Plotting of actual piece
                    if self.game.figure.y + i >= self.game.buffer:
                        pygame.draw.rect(self.screen, colors[self.game.figure.type],
                                         [self.game.x + self.game.zoom * (j + self.game.figure.x) + 1,
                                          self.game.y + self.game.zoom * (i + self.game.figure.y) + 1,
                                          self.game.zoom - 2, self.game.zoom - 2])


            # Plot SWAP piece if it has been set aside
            if self.game.swap_piece:
                for ind in self.game.swap_piece.image():
                    i = ind//4
                    j = ind%4
                    pygame.draw.rect(self.screen, colors[self.game.swap_piece.type],
                                     [self.game.swap_x + self.game.zoom * j,
                                      self.game.swap_y + self.game.zoom * i,
                                      self.game.zoom - 2, self.game.zoom - 2]) 
            else:
                # draw something indicating what this spot is for
                pass

            fig_i = 0
            for fig in self.game.queue:
                for ind in fig.image():
                    i = ind//4
                    j = ind%4
                    pygame.draw.rect(self.screen, colors[fig.type],
                                     [self.game.queue_x + self.game.zoom * j,
                                      self.game.queue_y + self.game.zoom * (i + fig_i*5) , # testing coordinates
                                      self.game.zoom - 2, self.game.zoom - 2]) 

                fig_i += 1



            # Displaying screen text
            font = pygame.font.SysFont('Calibri', 25, True, False)
            font1 = pygame.font.SysFont('Calibri', 65, True, False)
            text_score = font.render("Score: " + str(self.game.score), True, BLACK)
            text_lines = font.render("Lines: " + str(self.game.lines), True, BLACK)
            text_level = font.render("Level: " + str(self.game.level), True, BLACK)
            text_game_over = font1.render("Game Over", True, (255, 125, 0))
            text_game_over1 = font1.render("Press q", True, (255, 215, 0))
            text_swap = font.render("SWAP!", True, BLACK)
            text_queue = font.render("Queue:", True, BLACK)
            text_reward = font.render(f'Reward: {round(self.game.reward,2)}', True, BLACK)



            if self.game.player == 0:
                p = 'Human'
            else:
                p = 'Computer'
            text_player = font.render(f'{p}: \'p\' to swap', True, (200, 50, 100))     

            controlsX = 10
            controlsY = 300
            position = controlsX, controlsY
            font = pygame.font.SysFont('Calibri', 15)
            text_control = ["Controls",
                    "/: CCW rotation",
                   "rShift': CW rotation",
                   "up,down,left,right: movement",
                   "space: hard drop",
                   "0-6: debug blocks",
                   "s: swap",
                   "q: restart game",
                   "l: free line"]
            label = []
            for line in text_control: 
                label.append(font.render(line, True, GRAY))    
            for line in range(len(label)):
                self.screen.blit(label[line],(position[0],position[1]+(line*15)+(15*line)))

            self.screen.blit(text_player, [400, 50])
            self.screen.blit(text_score, [100, 50])
            self.screen.blit(text_lines, [100, 100])
            self.screen.blit(text_level, [100, 150])

            self.screen.blit(text_swap, [50, 250])
            self.screen.blit(text_queue, [self.game.queue_x, self.game.queue_y-50])
            self.screen.blit(text_reward, [0,0])
            # screen.blit(text_controls, [50, 300])
            if self.game.state == "gameover":
                self.screen.blit(text_game_over, [250, 80])
                self.screen.blit(text_game_over1, [250, 140])
            else:
                # update time if game is still going
                seconds=(pygame.time.get_ticks()-self.game_start_time)/1000
                text_timer = font.render(f'Time: {round(seconds)} s', True, BLACK)
                self.screen.blit(text_timer, [10, 50])



        #         start_ticks=pygame.time.get_ticks() #starter tick
        #      #calculate how many seconds
        #     if seconds>10: # if more than 10 seconds close the game
        #         break
        #     print (seconds) #print how many seconds

            pygame.display.flip()
            self.clock.tick(self.game.fps)
        pygame.quit()
        print('pygame quit!')

In [11]:
game = Game_env()

pygame quit!
